# Ranking Strategies Notebook
This notebook normalized cumulative, reliability, and decay metrics and ranks strategies highest to lowest based on these metrics.

## Import Libraries

In [1]:
from ipywidgets import interact, interact_manual
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import  util
from EquityHedging.analytics.util import get_df_weights
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from EquityHedging.reporting import formatter as fmt
from EquityHedging.analytics import hedge_metrics as hm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go


## Import returns data

### Imports Daily, Weekly, Monthly, Quarterly and Yearly returns data

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**) - SPTR, M1WD, SX5T
* Decide if you want a Fixed Income Benchmark (**include_fi**) - True, False
* Select what equity hedging strategies to exclude (**strat_drop_list**). Below is the list of strategies:

    * 99%/90% Put Spread, Down Var, Vortex, VOLA , Dynamic Put Spread, VRR, GW Dispersion, Corr Hedge

In [2]:
equity_bmk = 'M1WD'
include_fi = False
weighted = [True, False]
strat_drop_list = ['99%/90% Put Spread', 'Vortex']
new_strat = True
returns = dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

##  Get Notional Weights 

In [3]:
#notional_weights = dm.get_notional_weights(returns['Monthly'])
notional_weights = [19.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.25, 1.0]
df_weights = get_df_weights(notional_weights, list(returns['Monthly'].columns), include_fi)
fmt.get_notional_styler(df_weights)

,M1WD,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var
Notional Weights (Billions),$19.00,$1.00,$1.00,$1.00,$1.00,$1.00,$0.25,$1.00
Percentage Weights,100.00%,5.26%,5.26%,5.26%,5.26%,5.26%,1.32%,5.26%
Strategy Weights,0.00%,16.00%,16.00%,16.00%,16.00%,16.00%,4.00%,16.00%


## Get Hedge Metrics and Normalized Hedge Metrics

In [4]:
data = summary.get_normalized_hedge_metrics(returns, equity_bmk, notional_weights, weighted_hedge = True)
df_norm = data['Normalized Data']
df_hm = data['Hedge Metrics']


## Display Data
To display data:
* Select which statistics you want to view **(stats)** -  Hedge Metrics or Normalized Data

In [5]:
@interact
def display_hedge_metrics_or_normal( stats = ['Hedge Metrics','Normalized Data']):
    if stats == 'Hedge Metrics':
        return fmt.format_hm_to_normalize(df_hm)    
    if stats == 'Normalized Data':
        return fmt.format_normalized_data(df_norm)


interactive(children=(Dropdown(description='stats', options=('Hedge Metrics', 'Normalized Data'), value='Hedge…

## Assign Each Strategy a Symbol

In [6]:
df_normal = df_norm.transpose()
symbols = plots.get_symbols(df_normal, weighted_hedge = True)


## Display Ranked Strategies vs Normalized Data

In [19]:
#transpose normal data here instead of in method
index = list(df_normal.index)
fig = go.Figure()
#loops through eaach column and plots each metric for the given strategy
for col in df_normal.columns:
    fig.add_trace(go.Scatter(
        x=index,
        y=df_normal[col],
        marker=dict(size = 20),
        marker_symbol = symbols[col],
        mode= "markers",
        name=col
        ))
fig.update_layout(title="",
                xaxis_title="Hedge Metric",
                yaxis_title="Higher the Better",
                  plot_bgcolor='White')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
#fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='Black')
#fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='Black')
fig.show()

In [13]:
#pip install -U kaleido
#fig.write_image("fig1.png")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
